In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,66823
2,Huelva,Confirmados PDIA 14 días,7917
3,Huelva,Tasa PDIA 14 días,"1542,7636066021007"
4,Huelva,Confirmados PDIA 7 días,3547
5,Huelva,Tasa PDIA 7 dias,"691,1939513221739"
6,Huelva,Total Confirmados,67045
7,Huelva,Curados,51911
8,Huelva,Fallecidos,432


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  66823.0


/tmp/ipykernel_2905/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  19566.0


/tmp/ipykernel_2905/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_2905/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_2905/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_2905/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 121 personas en los últimos 7 días 

Un positivo PCR cada 53 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,66823.0,7917.0,3547.0,513170.0,691.193951,1542.763607,718.0
Huelva-Costa,39104.0,4492.0,1956.0,289548.0,675.535662,1551.383536,327.0
Sierra de Huelva-Andévalo Central,7785.0,1610.0,738.0,67391.0,1095.101720,2389.043047,213.0
Huelva (capital),19566.0,2697.0,1191.0,143837.0,828.020607,1875.039107,207.0
Condado-Campiña,19474.0,1803.0,850.0,156231.0,544.066158,1154.060334,180.0
Valverde del Camino,1709.0,331.0,128.0,12750.0,1003.921569,2596.078431,65.0
Gibraleón,1404.0,223.0,111.0,12737.0,871.476800,1750.804742,50.0
Almonte,2975.0,228.0,118.0,24507.0,481.495083,930.346432,29.0
Aljaraque,2640.0,382.0,159.0,21474.0,740.430288,1778.895408,28.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,28.0,23.0,2.0,330.0,606.060606,6969.696970,4.0
Cerro de Andévalo (El),363.0,109.0,59.0,2327.0,2535.453373,4684.142673,18.0
Rosal de la Frontera,197.0,71.0,29.0,1697.0,1708.898055,4183.853860,9.0
Jabugo,222.0,73.0,37.0,2260.0,1637.168142,3230.088496,15.0
Campillo (El),205.0,63.0,25.0,2024.0,1235.177866,3112.648221,7.0
Fuenteheridos,69.0,22.0,10.0,709.0,1410.437236,3102.961918,1.0
Nerva,511.0,156.0,81.0,5169.0,1567.034243,3017.991875,26.0
Zalamea la Real,279.0,92.0,38.0,3054.0,1244.269810,3012.442698,3.0
Cumbres Mayores,269.0,50.0,34.0,1749.0,1943.967982,2858.776444,7.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Hinojales,28.0,23.0,2.0,330.0,606.060606,6969.696970,4.0,0.086957
Alájar,58.0,8.0,2.0,759.0,263.504611,1054.018445,0.0,0.250000
Encinasola,198.0,32.0,8.0,1310.0,610.687023,2442.748092,1.0,0.250000
Paterna del Campo,393.0,42.0,11.0,3457.0,318.194967,1214.926237,2.0,0.261905
Villanueva de las Cruces,46.0,11.0,3.0,387.0,775.193798,2842.377261,0.0,0.272727
Villalba del Alcor,562.0,29.0,8.0,3366.0,237.670826,861.556744,1.0,0.275862
Corteconcepción,71.0,7.0,2.0,536.0,373.134328,1305.970149,0.0,0.285714
Villablanca,357.0,24.0,7.0,2885.0,242.634315,831.889081,2.0,0.291667
Villanueva de los Castillejos,302.0,36.0,11.0,2825.0,389.380531,1274.336283,1.0,0.305556
